In [300]:
import pandas as pd
import numpy as np

In [301]:
data = pd.read_excel("Session Planning LSC.xlsx")

In [302]:
IDs = data['Abstract #']
titles = data['Title']
keywords = data['Keywords']
abstracts = data['Abstract Summary']

In [303]:
all_title_words = []
for i in range(len(titles)):
    all_title_words = all_title_words + titles[i].split() + keywords[i].split()

In [304]:
# remove duplicates
all_title_words = list(dict.fromkeys(all_title_words))

In [305]:
# define stop words
stop_words = ['a', 'an', 'and', 'the', 'by', 'in', 'on', 'for', 'about', 'we', 'can', 'is', 'are', 'it', 'or', 'of', 'at', 
              'to', 'how', 'up', 'was', 'were', 'so', 'this', 'that', 'there', 'be', 'some', 'very', 'every', 
              'all', 'which', 'here', 'where', 'who', '-', 'under', 'with', 'without','same', 'one', 
             'based','using', 'study', 'problem', 'address','applied','cross']

In [306]:
# remove stop words, keep only stem of long words
title_words = []
for i in range(len(all_title_words)):
    this_word = all_title_words[i].lower()
    if this_word not in stop_words and len(this_word) > 3:
        if len(this_word) > 5:
            title_words = title_words + [this_word[0:5]]
        else:
            title_words = title_words + [this_word]
# dedup one more time
title_words = list(dict.fromkeys(title_words))
print(title_words)

['solvi', 'gener', 'share', 'mathe', 'algor', 'parti', 'formu', 'simul', 'annea', 'mutat', 'strat', 'bi-ob', 'optim', 'model', 'three', 'human', 'relie', 'chain', 'type', 'keywo', 'relay', 'full', 'truck', 'freig', 'trans', 'auton', 'route', 'short', 'dista', 'trave', 'predi', 'conta', 'water', 'traff', 'unite', 'barge', 'shipp', 'inlan', 'marit', 'suppl', 'resil', 'asses', 'lared', 'port', 'entry', 'durin', 'covid', 'pande', 'queue', 'theor', 'littl', 'p-ell', 'cover', 'drone', 'deliv', 'netwo', 'desig', 'facil', 'locat', 'area', 'ellip', 'np-ha', 'triti', 'analy', 'mini-', 'store', 'chara', 'barra', '(colo', 'logis', 'retai', 'data', 'minin', 'devel', 'neckl', 'float', 'life-', 'devic', 'produ', 'plan', 'innov', 'plann', 'susta', 'deep', 'reinf', 'learn', 'inven', 'peris', 'quant', 'disco', 'contr', 'manag', 'deman', 'disru', 'signa', 'track', 'inter', 'south', 'regio', 'u.s.', 'prici', 'decis', 'assor', 'neste', 'logit', 'choic', 'multi', 'newsv', 'autom', 'indus', 'food', 'crop', '

In [307]:
len(title_words)

361

In [308]:
# Compute the whole feature matrix
title_words_freq = [0 for ij in range(len(title_words))]
for i in range(len(titles)):
    this_doc = (titles[i] + keywords[i] + abstracts[i]).split()
    this_doc_words = []
    for j in range(len(this_doc)):
        this_word = this_doc[j].lower()
        if this_word not in stop_words and len(this_word) > 3:
            if len(this_word) > 5:
                this_word = this_word[0:5]
            this_doc_words = this_doc_words + [this_word]
    this_doc_words = list(dict.fromkeys(this_doc_words))  # dedup
    for j in range(len(title_words)):
        for k in range(len(this_doc_words)):
            if this_doc_words[k] == title_words[j]:
                title_words_freq[j] += 1


In [309]:
stem_freq = pd.DataFrame({"stem":title_words, "freq":title_words_freq})
stem_freq = stem_freq.sort_values(by = "freq",ascending=False)
stem_freq.to_csv("stem_freq_all.csv")

In [ ]:
# Now open the stem_freq_all.csv, eyeball all entries, and hand pick useful ones by adding a pick column
# Set the pick value to 1 for picked entries, save as stem_freq_all_pick.csv
# Then continue

In [310]:
# Hand pick key words
stem_pick = pd.read_csv("stem_freq_all_pick.csv")
title_words = list(stem_pick.loc[stem_pick['pick'] == 1, 'stem'])
feature_matrix = np.zeros((len(titles),len(title_words)))
feature_weight = [1 for i in range(len(title_words))]

In [311]:
# Compute the whole feature matrix
for i in range(len(titles)):
    this_doc = (titles[i] + keywords[i] + abstracts[i]).split()
    this_doc_words = []
    for j in range(len(this_doc)):
        this_word = this_doc[j].lower()
        if this_word not in stop_words and len(this_word) > 3:
            if len(this_word) > 5:
                this_word = this_word[0:5]
            this_doc_words = this_doc_words + [this_word]
    this_doc_words = list(dict.fromkeys(this_doc_words))  # dedup
    for j in range(len(title_words)):
        for k in range(len(this_doc_words)):
            if this_doc_words[k] == title_words[j]:
                feature_matrix[i,j] = 1

In [312]:
feature_matrix.shape

(63, 98)

In [313]:
# Compute the doc-doc similarity matrix
simi_matrix = np.zeros((len(titles),len(titles)))

In [314]:
# Compute the distance matrix (upper triangular)
for i in range(len(titles)):
    for j in range(i+1, len(titles)-1):
        sij = np.multiply(feature_matrix[i,:], feature_matrix[j,:])
        sij = np.multiply(sij, feature_weight)
        simi_matrix[i,j] = np.sum(sij)

In [315]:
simi_matrix

array([[0., 2., 2., ..., 1., 1., 0.],
       [0., 0., 2., ..., 3., 0., 0.],
       [0., 0., 0., ..., 2., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [217]:
# Now that we have the doc-to-doc similarity matrix, we can run an optimization model to group abstracts 
# Constraints: each session must have no more than 4 talks, fill all four slots if possible
# The score of a session is the sum of the pairwise similarity values of all talks placed in the session
# Objective: maximize the total score of the track (all sessions)
# This is a quadratic assignment problem.


In [316]:
# How much time do we give the solver?
reslim = 15*60  # 15 minutes
solver = 'cplex'

In [317]:
# Generate a GAMS model
with open("assign.gms","w") as f:
    f.write("option reslim = " + str(reslim) + ";\n")
    f.write("option miqcp = " + solver + ";\n")
    f.write("set i /1*" + str(len(titles)) +"/;\n")
    f.write("set g /1*" + str(int(np.ceil(len(titles)/ 4))) + "/;\n")
    f.write("alias(i,j,k);\nparameter s(i,j);\n")
    for i in range(len(titles)):
        for j in range(i+1, len(titles)-1):
            f.write("s('" + str(i+1) + "','" + str(j+1) + "') = " + str(simi_matrix[i,j]) + ";\n")
    model_str = '''
binary variables
x(i,g) 1 if talk i is in session g
;
variable group_score(g);
variable total_score;
equations
    eq_group_size(g) each group has at most four talks
    eq_score(g) calculate group score
    eq_place(i) every talk must be placed in a session
    eq_obj calculate objective value
;
eq_group_size(g)..
    sum(i, x(i,g)) =l= 4;
eq_place(i)..
    sum(g, x(i,g)) =e= 1;
eq_score(g)..
    group_score(g) =e= sum((i,j)$(ord(i)<ord(j)), s(i,j)*x(i,g)*x(j,g));
eq_obj..
    total_score =e= sum(g, group_score(g));
model assign /eq_group_size, eq_place, eq_score, eq_obj/;
solve assign maximize total_score using miqcp;
parameter result(i);
loop((i,g)$x.l(i,g),
    result(i) = ord(g);
);
display result;

file f /'sol.txt'/;
put f;
loop(i,
    put (ord(i)-1):0:0 ' ' (result(i)-1):0:0 /;
)
putclose;

    '''
    f.write(model_str)
    f.write("\n")
    

In [318]:
# Run the GAMS model
import subprocess
subprocess.run(["gams", "assign.gms"])

CompletedProcess(args=['gams', 'assign.gms'], returncode=0)

In [328]:
# Read the GAMS solution
sol = pd.read_csv("sol.txt", sep = " ", header = None)

In [329]:
res = pd.DataFrame({"Abstract ID":IDs, "Session ID":sol.iloc[:,1], "Title":titles, "Keyword":keywords, "Abstract":abstracts})
res

,Abstract ID,Session ID,Title,Keyword,Abstract
0,1101,15,Solving the general share-a-ride problem by a ...,matheuristic\ngeneral share-a-ride problem\nse...,\nThis research evaluates the General Share-a-...
1,3250,4,A Bi-Objective Optimization Model for a Three-...,TYPE NEW KEYWORD HERE,\nThe unpredictable nature and global impact o...
2,3533,7,Relay Model for Full Truckload Freight Transpo...,Relay\nSimulation\nTrucking,\nThe driver retention problem in the commerci...
3,3539,12,Optimizing Autonomous Trucking Routes for Shor...,Autonomous\nOptimization\nTrucking\nShort Dist...,\nAutonomous trucking is an innovative technol...
4,2024,1,Predicting Container-on-Barge Waterway Traffic...,Barge Transportation\nContainer Shipping\nCont...,\nContainer-on-Barge (COB) employs barges to m...
...,...,...,...,...,...
58,3430,4,Resiliency assessment of supply chain system d...,Resiliency\nsupply chain\nMarkovian process,\nThe effect of changing the environmental con...
59,1159,2,Blockchain Technology and Supply Chain Mapping...,Supply Chain Management\nSupply Chain Mapping\...,\nCOVID-19 pandemic led supply chain (SC) disr...
60,1984,10,A Carbon-Conscious Closed Loop Multi Objective...,Closed Loop Supply Chain\nSustainability\nMixe...,\nWith increasing demand for environmentally-f...
61,2308,9,Supply Chain Risk Management and Chain Resilience,supply chain risk\nchain resilience\nrisk mana...,\nThe supply chain risk can be expressed as a ...


In [330]:
# Extract the common word stems in each session
n_sessions = int(np.ceil(len(titles)/ 4))
stem2 = []  # stems that are in two or more abstracts
stem3 = []  # stems that are in three or more abstracts
for g in range(n_sessions):
    ids = []
    for i in range(len(titles)):
        if sol.iloc[i,1] == g:
            ids += [i]
    this_session_features = feature_matrix[ids,:]
    n = this_session_features.shape[0]
    common2_idx = []
    common3_idx = []
    for j in range(len(title_words)):
        freq = 0
        for k in range(n):
            if this_session_features[k,j] == 1:
                freq += 1;
        if freq >= 2:
            common2_idx += [j]
        if freq >= 3:
            common3_idx += [j]
    common_stem2 = ' '.join([title_words[i] for i in common2_idx])
    common_stem3 = ' '.join([title_words[i] for i in common3_idx])
    stem2 += [common_stem2]
    stem3 += [common_stem3]
session_summary = pd.DataFrame({"Session ID":[i for i in range(n_sessions)], "stem2":stem2, "stem3":stem3})
session_summary.to_csv("session_summary.csv")

In [331]:
session_summary

,Session ID,stem2,stem3
0,0,netwo deliv integ algor locat simul resil rout...,netwo algor
1,1,netwo const facil plann susta water,plann susta
2,2,netwo pande uncer disru predi,netwo pande predi
3,3,capac plann retai quali priva,
4,4,simul resil pande capac disru borde,simul resil disru
5,5,netwo deliv integ const capac disru mixed stoc...,integ const
6,6,netwo deliv integ const locat route capac rout...,deliv route capac routi mixed vehic linea food...
7,7,simul fleet marit,simul
8,8,netwo locat cover heuri np-ha,locat
9,9,facil resil uncer disas alloc relie stoch two-...,uncer stoch two-s


In [327]:
res.to_csv("result.csv")